In [1]:
"""
object.__getattr__(self, name):

    1. Called when an attribute lookup has NOT found the attribute in the usual places 
       (i.e. it is not an instance attribute nor is it found in the class tree for self). 
       This is an intentional asymmetry between __getattr__() and __setattr__()
    2. name is the attribute name. This method should return the (computed) attribute value 
       or raise an AttributeError exception.
       
object.__setattr__(self, name, value):

    1. Called when an attribute assignment is attempted. 
    2. This is called instead of the normal mechanism 
       (i.e. store the value in the instance dictionary). 
    3. name is the attribute name, value is the value to be assigned to it.
    4. If __setattr__() wants to assign to an instance attribute, 
       it should NOT simply execute 
       
           self.name = value 
           
       — this would cause a recursive call to itself. 
       Instead, it should insert the value in the dictionary of instance attributes, e.g., 
       
           self.__dict__[name] = value
           
       For new-style classes, rather than accessing the instance dictionary, 
       it should call the base class method with the same name, for example, 
       
           object.__setattr__(self, name, value).

object.__delattr__(self, name)

    1. Like __setattr__() but for attribute deletion instead of assignment. 
    2. This should only be implemented if del obj.name is meaningful for the object

object.__getattribute__(self, name):

    1. only for new-style classes
    2. Called UNConditionally to implement attribute accesses for instances of the class. 
    3. If the class also defines __getattr__(), the latter will NOT be called 
       unless __getattribute__() either calls it explicitly or 
       raises an AttributeError. 
    4. This method should return the (computed) attribute value or raise an AttributeError exception. 
    5. In order to avoid infinite recursion in this method, its implementation should 
       always call the base class method with the same name to access any attributes it needs, 
       for example, 
       
           object.__getattribute__(self, name)
           
    6. This method may still be bypassed when looking up special methods 
       as the result of implicit invocation via language syntax or built-in functions. 
       See:
          Special method lookup for new-style classes.
"""
print

In [4]:
class MyClass(object):

    def __init__(self):
        # prevents infinite recursion from self.data = {'a': 'v1', 'b': 'v2'}
        # as now we have __setattr__, which will call 
        #
        #     __getattr__ 
        #
        # when the line 
        #
        #     self.data[k] 
        #
        # tries to access 
        # 
        #     self.data
        # 
        # won't find it in the instance dictionary 
        # and return self.data[k] will in turn call __getattr__
        # for the same reason and so on.... 
        # so we manually set data initially
        super(MyClass, self).__setattr__('data', {'a': 'v1', 'b': 'v2'})

    def __setattr__(self, k, v):
        self.data[k] = v

    def __getattr__(self, k):
        # we don't need a special call to super here because getattr is only 
        # called when an attribute is NOT found in the instance's dictionary
        try:
            return self.data[k]
        except KeyError:
            raise AttributeError
            
ob = MyClass()
ob.c = 11
print ob.a, ob.b, ob.c
try:
    ob.d
except Exception as e:
    print type(e), e

v1 v2 11
<type 'exceptions.AttributeError'> 


In [22]:
"""demonstrate the recursive call"""
class MyClass(object):
    
    COUNT = 0
    
    def __init__(self):
        MyClass.COUNT += 1
        print MyClass.COUNT, '__init__'
        self.data = {'a': 'v1', 'b': 'v2'} # call __setattr__

    def __getattr__(self, attr):
        MyClass.COUNT += 1
        if MyClass.COUNT>3:
            print 'STOP calling __getattr__'
            return 'STOP calling __getattr__'
        print MyClass.COUNT, '__getattr__', self, self.__dict__
        return self.data[attr] # self.data will invoke self.__getattr__ to find data => infinite recursion
    
    def __setattr__(self, k, v):
        MyClass.COUNT += 1
        print MyClass.COUNT, '__setattr__'
        self.data[k] = v # call __getattr__
        
ob = MyClass()
ob.c = 11

1 __init__
2 __setattr__
3 __getattr__ <__main__.MyClass object at 0x0000000003A3EB70> {}
STOP calling __getattr__


TypeError: string indices must be integers, not str

In [40]:
"""another kind of fix"""
class MyClass(object):
    
    COUNT = 0
    
    def __init__(self):
        MyClass.COUNT += 1
        print MyClass.COUNT, '__init__'
        self.data = {'a': 'v1', 'b': 'v2'} # call __setattr__

    def __getattr__(self, attr):
        MyClass.COUNT += 1
        print MyClass.COUNT, '__getattr__', self, self.__dict__
        return self.data[attr] # self.data will invoke self.__getattr__ to find data => infinite recursion
    
    def __setattr__(self, k, v):
        MyClass.COUNT += 1
        if k=='data' and k not in self.__dict__:
            self.__dict__[k]=v
            return
        print MyClass.COUNT, '__setattr__', self.__dict__
        self.data[k] = v # call __getattr__
        
ob = MyClass()
ob.c = 11
ob.a, ob.b, ob.c

1 __init__
3 __setattr__ {'data': {'a': 'v1', 'b': 'v2'}}
4 __getattr__ <__main__.MyClass object at 0x00000000038EE7B8> {'data': {'a': 'v1', 'c': 11, 'b': 'v2'}}
5 __getattr__ <__main__.MyClass object at 0x00000000038EE7B8> {'data': {'a': 'v1', 'c': 11, 'b': 'v2'}}
6 __getattr__ <__main__.MyClass object at 0x00000000038EE7B8> {'data': {'a': 'v1', 'c': 11, 'b': 'v2'}}


('v1', 'v2', 11)